# Import data, compute features, train-val split, feature selection, pre-process, & save
## Import
### Modules

In [6]:
from __future__ import division
%matplotlib inline
import sys
sys.path.append('../')
from Modules.Basics import *
from ML_Tools.General.Feature_Selection import *
from ML_Tools.Transformations.HEP_Proc import *
import uproot

/home/giles/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/giles/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


## Feature processing

In [12]:
rotate = False
cartesian = False
useTrackFeatures = True
useTowerFeatures = True
features = ['jetPt',
 'jetEta',
 'jetMass',
 'ntracks',
 'ntowers']

normIn = True
pca = False

In [45]:
def trackFeatures(inData):
    inData["min_track_pT"] = -1
    inData.loc[inData.ntracks > 0, "min_track_pT"] = inData.loc[inData.ntracks > 0, "trackPt"].apply(np.min)
    
    inData["mean_track_pT"] = -1
    inData.loc[inData.ntracks > 0, "mean_track_pT"] = inData.loc[inData.ntracks > 0, "trackPt"].apply(np.mean)
    
    inData["max_track_pT"] = -1
    inData.loc[inData.ntracks > 0, "max_track_pT"] = inData.loc[inData.ntracks > 0, "trackPt"].apply(np.max)
    
    inData["sum_track_pT"] = -1
    inData.loc[inData.ntracks > 0, "sum_track_pT"] = inData.loc[inData.ntracks > 0,"trackPt"].apply(np.sum)
    
    inData["abs_track_eta"] = -1
    inData.loc[inData.ntracks > 0, "abs_track_eta"] = inData.loc[inData.ntracks > 0, "trackEta"].apply(np.abs)
    
    inData["min_track_eta"] = -1
    inData.loc[inData.ntracks > 0, "min_track_eta"] = inData.loc[inData.ntracks > 0, "abs_track_eta"].apply(np.min)
    
    inData["mean_track_eta"] = -1
    inData.loc[inData.ntracks > 0, "mean_track_eta"] = inData.loc[inData.ntracks > 0, "trackEta"].apply(np.mean)
    
    inData["max_track_eta"] = -1
    inData.loc[inData.ntracks > 0, "max_track_eta"] = inData.loc[inData.ntracks > 0, "abs_track_eta"].apply(np.max)
    
    inData["mean_track_charge"] = -1
    inData.loc["mean_track_charge"] = inData.loc[inData.ntracks > 0, "trackCharge"].apply(np.mean)
    

In [46]:
def towerFeatures(inData):
    inData["min_tower_E"] = -1
    inData.loc[inData.ntowers > 0, "min_tower_E"] = inData.loc[inData.ntowers > 0, "towerE"].apply(np.min)
    
    inData["mean_tower_E"] = -1
    inData.loc[inData.ntowers > 0, "mean_tower_E"] = inData.loc[inData.ntowers > 0, "towerE"].apply(np.mean)
    
    inData["max_tower_E"] = -1
    inData.loc[inData.ntowers > 0, "max_tower_E"] = inData.loc[inData.ntowers > 0, "towerE"].apply(np.max)
    
    inData["sum_tower_E"] = -1
    inData.loc[inData.ntowers > 0, "sum_tower_E"] = inData.loc[inData.ntowers > 0, "towerE"].apply(np.sum)
    
    inData["min_tower_Eem"] = -1
    inData.loc[inData.ntowers > 0, "min_tower_Eem"] = inData.loc[inData.ntowers > 0, "towerEem"].apply(np.min)
    
    inData["mean_tower_Eem"] = -1
    inData.loc[inData.ntowers > 0, "mean_tower_Eem"] = inData.loc[inData.ntowers > 0, "towerEem"].apply(np.mean)
    
    inData["max_tower_Eem"] = -1
    inData.loc[inData.ntowers > 0, "max_tower_Eem"] = inData.loc[inData.ntowers > 0, "towerEem"].apply(np.max)
    
    inData["sum_tower_Eem"] = -1
    inData.loc[inData.ntowers > 0, "sum_tower_Eem"] = inData.loc[inData.ntowers > 0, "towerEem"].apply(np.sum)
    
    inData["min_tower_Ehad"] = -1
    inData.loc[inData.ntowers > 0, "min_tower_Ehad"] = inData.loc[inData.ntowers > 0, "towerEhad"].apply(np.min)
    
    inData["mean_tower_Ehad"] = -1
    inData.loc[inData.ntowers > 0,"mean_tower_Ehad"] = inData.loc[inData.ntowers > 0,"towerEhad"].apply(np.mean)
    
    inData["max_tower_Ehad"] = -1
    inData.loc[inData.ntowers > 0,"max_tower_Ehad"] = inData.loc[inData.ntowers > 0,"towerEhad"].apply(np.max)
    
    inData["sum_tower_Ehad"] = -1
    inData.loc[inData.ntowers > 0,"sum_tower_Ehad"] = inData.loc[inData.ntowers > 0,"towerEhad"].apply(np.sum)
    
    inData["tower_Eem_frac"] = -1
    inData.loc[inData.ntowers > 0,"tower_Eem_frac"] = inData.loc[inData.ntowers > 0, "sum_tower_Eem"]/inData.loc[inData.ntowers > 0, "sum_tower_E"]
    
    inData["tower_Ehad_frac"] = -1
    inData.loc[inData.ntowers > 0,"tower_Ehad_frac"] = inData.loc[inData.ntowers > 0, "sum_tower_Ehad"]/inData.loc[inData.ntowers > 0, "sum_tower_E"]
    
    inData["min_tower_eta"] = -1
    inData.loc[inData.ntowers > 0,"min_tower_eta"] = inData.loc[inData.ntowers > 0,"towerEta"].apply(np.min)
    
    inData["mean_tower_eta"] = -1
    inData.loc[inData.ntowers > 0,"mean_tower_eta"] = inData.loc[inData.ntowers > 0,"towerEta"].apply(np.mean)
    
    inData["max_tower_eta"] = -1
    inData.loc[inData.ntowers > 0,"max_tower_eta"] = inData.loc[inData.ntowers > 0,"towerEta"].apply(np.max)

In [100]:
def getBatch(sample, mode, batch, nBatches):
    f = uproot.open(dirLoc + sample + '_' + mode + '.root')['treeJets']
    totalSize = len(f)
    batchSize = math.floor(totalSize/nBatches)
    data = f.pandas.df(entrystart=batchSize*batch, entrystop=batchSize*(batch+1))
    if useTowerFeatures: towerFeatures(data)
    if useTrackFeatures: trackFeatures(data)
    data.drop(columns=['trackPt', 'trackEta', 'trackPhi', 'trackCharge', 'towerE','towerEem', 'towerEhad', 'towerEta', 'towerPhi', 'abs_track_eta'], inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    #print "NaNs", data.columns[data.isnull().any()].tolist()
    data.dropna(inplace=True) 
    return data

##  Feature selection

In [88]:
gluons = getBatch('gluons', 'standard', 75, 100)
quarks = getBatch('quarks', 'standard', 75, 100)
gluons['gen_target'] = getTarget('gluon')
quarks['gen_target'] = getTarget('quark')
batch = gluons.append(quarks, ignore_index=True)
batch = batch.sample(frac=1).reset_index(drop=True)

NaNs ['jetPt', 'jetEta', 'jetPhi', 'jetMass', 'ntracks', 'ntowers', 'min_tower_E', 'mean_tower_E', 'max_tower_E', 'sum_tower_E', 'min_tower_Eem', 'mean_tower_Eem', 'max_tower_Eem', 'sum_tower_Eem', 'min_tower_Ehad', 'mean_tower_Ehad', 'max_tower_Ehad', 'sum_tower_Ehad', 'tower_Eem_frac', 'tower_Ehad_frac', 'min_tower_eta', 'mean_tower_eta', 'max_tower_eta', 'min_track_pT', 'mean_track_pT', 'max_track_pT', 'sum_track_pT', 'min_track_eta', 'mean_track_eta', 'max_track_eta', 'mean_track_charge']
NaNs ['jetPt', 'jetEta', 'jetPhi', 'jetMass', 'ntracks', 'ntowers', 'min_tower_E', 'mean_tower_E', 'max_tower_E', 'sum_tower_E', 'min_tower_Eem', 'mean_tower_Eem', 'max_tower_Eem', 'sum_tower_Eem', 'min_tower_Ehad', 'mean_tower_Ehad', 'max_tower_Ehad', 'sum_tower_Ehad', 'tower_Eem_frac', 'tower_Ehad_frac', 'min_tower_eta', 'mean_tower_eta', 'max_tower_eta', 'min_track_pT', 'mean_track_pT', 'max_track_pT', 'sum_track_pT', 'min_track_eta', 'mean_track_eta', 'max_track_eta', 'mean_track_charge']


In [89]:
batch.head()

,jetPt,jetEta,jetPhi,jetMass,ntracks,ntowers,min_tower_E,mean_tower_E,max_tower_E,sum_tower_E,...,max_tower_eta,min_track_pT,mean_track_pT,max_track_pT,sum_track_pT,min_track_eta,mean_track_eta,max_track_eta,mean_track_charge,gen_target
0,114.612373,0.718398,1.011648,5.576128,10.0,7.0,2.501653,6.369087,14.534379,44.583607,...,0.861179,0.641598,7.987243,22.188534,79.872429,0.560613,0.723994,0.840256,-1.0,1
1,125.804237,2.572583,-1.357541,13.253658,8.0,16.0,1.213994,15.081958,79.933884,241.311325,...,2.867692,1.170898,11.177648,43.335938,89.421181,2.541731,2.622050,2.841227,-1.0,0
2,107.762634,-3.080965,0.301674,7.068763,6.0,14.0,0.637240,25.146957,94.462532,352.057404,...,-2.767432,1.114041,12.559344,38.020153,75.356064,3.007479,-3.085623,3.201178,-1.0,0
3,108.488312,0.458741,-2.507700,6.943421,3.0,9.0,1.636533,9.755098,28.905098,87.795883,...,0.497438,1.240126,9.774052,21.865152,29.322155,0.459853,0.473810,0.501062,-1.0,1
4,102.518318,0.549075,2.390665,4.504710,6.0,2.0,0.745537,0.960165,1.174792,1.920329,...,0.733376,2.577368,16.830477,56.558506,100.982864,0.419189,0.520883,0.607653,-1.0,1


In [91]:
%%time

importantFeatures = rankClassifierFeatures(batch, [x for x in batch.columns if 'gen' not in x])

Running fold 1 /10
ROC AUC: 0.78294
Running fold 2 /10
ROC AUC: 0.79956
Running fold 3 /10
ROC AUC: 0.79191
Running fold 4 /10
ROC AUC: 0.79062
Running fold 5 /10
ROC AUC: 0.79204
Running fold 6 /10
ROC AUC: 0.78798
Running fold 7 /10
ROC AUC: 0.79164
Running fold 8 /10
ROC AUC: 0.79049
Running fold 9 /10
ROC AUC: 0.78839
Running fold 10 /10
ROC AUC: 0.79761
29 important features identified
Feature	Importance
---------------------
ntracks 	0.14472443908452987
ntowers 	0.13109368458390236
jetMass 	0.12708177790045738
jetPt 	0.05909232422709465
max_track_pT 	0.0560832928866148
max_track_eta 	0.05449230708181858
min_track_eta 	0.04848376028239727
sum_tower_Ehad 	0.04618818275630474
max_tower_E 	0.03442402929067612
sum_track_pT 	0.031842540204525
mean_tower_Eem 	0.03169454745948315
mean_track_pT 	0.027389725483953953
tower_Eem_frac 	0.024386080354452132
min_track_pT 	0.023666434548795225
sum_tower_Eem 	0.021944370400160552
min_tower_E 	0.020363260060548782
jetPhi 	0.015202581556513906
sum_

In [95]:
features = importantFeatures[0]
print features

['ntracks', 'ntowers', 'jetMass', 'jetPt', 'max_track_pT', 'max_track_eta', 'min_track_eta', 'sum_tower_Ehad', 'max_tower_E', 'sum_track_pT', 'mean_tower_Eem', 'mean_track_pT', 'tower_Eem_frac', 'min_track_pT', 'sum_tower_Eem', 'min_tower_E', 'jetPhi', 'sum_tower_E', 'min_tower_eta', 'tower_Ehad_frac', 'max_tower_Eem', 'mean_tower_E', 'max_tower_Ehad', 'max_tower_eta', 'mean_tower_eta', 'jetEta', 'mean_tower_Ehad', 'mean_track_eta', 'min_tower_Eem']


## Save data

In [96]:
def getPipe(inData):
    inputPipe, outputPipe = getPreProcPipes(normIn=normIn, pca=pca)
    inputPipe.fit(inData[features].values.astype('float32'))
    
    with open(dirLoc + 'inputPipe.pkl', 'w') as fout:    
        pickle.dump(inputPipe, fout)
        
    return inputPipe

In [97]:
def saveBatch(inData, n, inputPipe, outFile):
    grp = outFile.create_group('fold_' + str(n))
    
    X = inputPipe.transform(inData[features].values.astype('float32'))
    inputs = grp.create_dataset("inputs", shape=X.shape, dtype='float32')
    inputs[...] = X
    
    y = inData['gen_target'].values.astype('int')
    targets = grp.create_dataset("targets", shape=y.shape, dtype='int')
    targets[...] = y

In [104]:
def prepareSample(sample,  mode, inputPipe, nSplit=10, nSave=10):
    print "Running", mode
    os.system('rm ' + dirLoc + mode + '.hdf5')
    outFile = h5py.File(dirLoc + mode + '.hdf5', "w")

    for i in xrange(nSave):
        gluons = getBatch('gluons', sample, i, nSplit)
        quarks = getBatch('quarks', sample, i, nSplit)
        gluons['gen_target'] = getTarget('gluon')
        quarks['gen_target'] = getTarget('quark')
        batch = gluons.append(quarks, ignore_index=True)
        batch = batch.sample(frac=1).reset_index(drop=True) #Shuffle
        
        if isinstance(inputPipe, types.NoneType):
            print "Fitting inputPipe"
            inputPipe = getPipe(batch)
        
        print "Saving fold:", i, "of", nSave, "events"
        saveBatch(batch, i, inputPipe, outFile)
        
    return inputPipe

In [105]:
inputPipe = prepareSample('standard', 'train', None, nSplit=100, nSave=10)
prepareSample('modified', 'testing', inputPipe, nSplit=100, nSave=10)

Running train
Fitting inputPipe
Saving fold: 0 of 10 events
Saving fold: 1 of 10 events
Saving fold: 2 of 10 events
Saving fold: 3 of 10 events
Saving fold: 4 of 10 events
Saving fold: 5 of 10 events
Saving fold: 6 of 10 events
Saving fold: 7 of 10 events
Saving fold: 8 of 10 events
Saving fold: 9 of 10 events
Running testing
Saving fold: 0 of 10 events
Saving fold: 1 of 10 events
Saving fold: 2 of 10 events
Saving fold: 3 of 10 events
Saving fold: 4 of 10 events
Saving fold: 5 of 10 events
Saving fold: 6 of 10 events
Saving fold: 7 of 10 events
Saving fold: 8 of 10 events
Saving fold: 9 of 10 events


Pipeline(memory=None,
     steps=[('normIn', StandardScaler(copy=True, with_mean=True, with_std=True))])